In [1]:
import datetime as dt
import json
from pprint import pprint

import pandas as pd
import requests

import config

In [2]:
WORKSPACE_ID = config.WORKSPACE_ID
USER_ID = config.USER_ID
TOKEN_VALUE = config.TOKEN_VALUE
API_KEY = config.API_KEY
API_SECRET = config.API_SECRET

In [3]:
headers = {
    "workspaceId": WORKSPACE_ID,
    "userId": USER_ID,
    "tokenValue": TOKEN_VALUE,
    "apiKey": API_KEY,
    "apiSecret": API_SECRET,
}

## Get number of box items

In [4]:
url = "https://server.itemit.com/items/itemit/v4/count"

filters = {
    "allOf": [
        {
            "anyOf": [
                {
                    "where": "COLLECTION",
                    "has": {"name": "Drop-Off Point Box"},
                },
                {
                    "where": "COLLECTION",
                    "has": {"name": "Delivery Box - Red"},
                },
            ]
        },
        {"allOf": []},
        {"noneOf": []},
    ]
}

payload = {
    "search": "",
    "filters": filters,
    "sorts": [],
}

response = requests.request("POST", url, headers=headers, data=json.dumps(payload))

In [5]:
size = response.json()
assert type(size) == int, "size response should be of type int"

In [6]:
print(size)

195


## Get all box items

In [7]:
url = "https://server.itemit.com/items/itemit/v7/profiles/_search"

sorts = [{"sort": "ITEM", "by": {"name": "ASC"}}]

payload = {
    "size": size,
    "page": 1,
    "search": "",
    "filters": filters,
    "sorts": sorts,
}

response = requests.request("POST", url, headers=headers, data=json.dumps(payload))

In [8]:
items = response.json()

In [9]:
pprint(items[0])

{'_deprecated_': False,
 '_expandedDate': '2021-03-12T18:59:58.025Z',
 '_source': {'created': {'_user': {'id': '944e89dc-e4b8-6854-44bb-0000ce3064bd'},
                         'client': {'apiVersion': 1.0, 'platform': 'SERVER'},
                         'datetime': '2020-09-25T14:56:14.071Z'},
             'description': None,
             'id': 'dfd29d7d-aef2-494d-8ae1-1607d4b97531',
             'lastSeen': {'_user': {'email': 'cu.covid.testing.4@gmail.com',
                                    'firstName': 'Covid',
                                    'id': 'ccc53756-4506-4685-934d-09f92d8e6cef',
                                    'lastName': 'Testing',
                                    'revision': 1,
                                    'type': 'USER'},
                          'address': 'Location not captured',
                          'coordinates': None,
                          'datetime': '2021-03-10T13:07:38.250Z',
                          'method': 'SCAN_QR'},
        

In [10]:
item = items[0]

pprint(item)

{'_deprecated_': False,
 '_expandedDate': '2021-03-12T18:59:58.025Z',
 '_source': {'created': {'_user': {'id': '944e89dc-e4b8-6854-44bb-0000ce3064bd'},
                         'client': {'apiVersion': 1.0, 'platform': 'SERVER'},
                         'datetime': '2020-09-25T14:56:14.071Z'},
             'description': None,
             'id': 'dfd29d7d-aef2-494d-8ae1-1607d4b97531',
             'lastSeen': {'_user': {'email': 'cu.covid.testing.4@gmail.com',
                                    'firstName': 'Covid',
                                    'id': 'ccc53756-4506-4685-934d-09f92d8e6cef',
                                    'lastName': 'Testing',
                                    'revision': 1,
                                    'type': 'USER'},
                          'address': 'Location not captured',
                          'coordinates': None,
                          'datetime': '2021-03-10T13:07:38.250Z',
                          'method': 'SCAN_QR'},
        

In [11]:
print("Current locations of all Drop-Off Point Boxes\n".upper())
print(
    f"No.\t{'Name':12s}\t{'Collection':12s}\t{'Location':22s}\t{'Last Seen Time':20s}\t{'Last Seen By':20s}"
)
print(
    f"---\t{'----':12s}\t{'--------':12s}\t{'--------------':20s}\t{'------------':20s}"
)

data = {
    "name": [],
    "collection": [],
    "location": [],
    "last_seen_dt": [],
    "last_seen_by": [],
    "last_seen_by_email": [],
}

for i, item in enumerate(items):
    name = item["_source"]["name"]
    collection = [
        element["_source"]["name"]
        for element in item["parentObjects"]["elements"]
        if element["_source"]["collectionType"] == "COLLECTION"
    ]
    collection = next(iter(collection), None)

    location = [
        element["_source"]["name"]
        for element in item["parentObjects"]["elements"]
        if element["_source"]["collectionType"] == "LOCATION"
    ]
    location = next(iter(location), None)

    try:
        last_seen_time = item["_source"]["lastSeen"]["datetime"]
        last_seen_time = pd.to_datetime(last_seen_time)
    except TypeError:
        last_seen_time = None

    try:
        first_name = item["_source"]["lastSeen"]["_user"]["firstName"]
    except TypeError:
        first_name = ""
    try:
        last_name = item["_source"]["lastSeen"]["_user"]["lastName"]
    except TypeError:
        last_name = ""
    last_seen_by = f"{first_name} {last_name}"
    last_seen_by = last_seen_by if last_seen_by != " " else None
    
    try:
        last_seen_by_email = item["_source"]["lastSeen"]["_user"]["email"]
    except TypeError:
        last_seen_by_email = None

    data["name"].append(name)
    data["collection"].append(collection)
    data["location"].append(location)
    data["last_seen_dt"].append(last_seen_time)
    data["last_seen_by"].append(last_seen_by)
    data["last_seen_by_email"].append(last_seen_by_email)

    print(
        f"{i:2d}.\t{data['name'][i]:12.12s}\t{str(data['collection'][i]):12.12s}\t{str(data['location'][i]):22.22s}\t{str(data['last_seen_dt'][i]):20.20s}\t{str(data['last_seen_by'][i]):20.20s}"
        #         f"{i:2d}.\t{data['name'][i]:12.12s}\t{data['location'][i]:22.22s}\t{data['last_seen_dt'][i]:20.20s}\t{data['last_seen_by'][i]:20.20s}"
    )

CURRENT LOCATIONS OF ALL DROP-OFF POINT BOXES

No.	Name        	Collection  	Location              	Last Seen Time      	Last Seen By        
---	----        	--------    	--------------      	------------        
 0.	CAI-1-A     	Delivery Box	Gonville & Caius Colle	2021-03-10 13:07:38.	Covid Testing       
 1.	CAI-1-B     	Delivery Box	Box Storage Room      	2021-03-11 09:41:30.	Covid Testing       
 2.	CAI-2-A     	Delivery Box	Gonville & Caius Colle	2021-03-10 13:07:40.	Covid Testing       
 3.	CAI-2-B     	Delivery Box	None                  	None                	None                
 4.	CAI-3-A     	Delivery Box	Gonville & Caius Colle	2021-03-10 13:07:42.	Covid Testing       
 5.	CAI-3-B     	Delivery Box	Box Storage Room      	2021-03-11 09:46:13.	Covid Testing       
 6.	CC-1-A      	Delivery Box	Corpus Christi Main Pl	2021-03-08 12:41:08.	Covid Testing       
 7.	CC-1-B      	Delivery Box	Box Storage Room      	2021-03-09 11:19:29.	Covid Testing       
 8.	CC-2-A      	Delivery 

In [17]:
df = pd.DataFrame(data)

In [18]:
# pd.set_option("display.max_rows", None)
# pd.reset_option("display.max_rows")
df.sort_values("last_seen_dt", ascending=False)

,name,collection,location,last_seen_dt,last_seen_by,last_seen_by_email
84,JE-P-1-B,Drop-Off Point Box,Anne McLaren Building,2021-03-12 13:58:31.839000+00:00,Covid Testing,cu.covid.testing.1@gmail.com
104,JN-P-2-B,Drop-Off Point Box,Anne McLaren Building,2021-03-12 13:58:29.437000+00:00,Covid Testing,cu.covid.testing.1@gmail.com
131,N-P-1-B,Drop-Off Point Box,Anne McLaren Building,2021-03-12 13:58:26.244000+00:00,Covid Testing,cu.covid.testing.1@gmail.com
45,ENG-P-1-B,Drop-Off Point Box,Anne McLaren Building,2021-03-12 13:58:24.367000+00:00,Covid Testing,cu.covid.testing.1@gmail.com
50,ENG-P-2-B,Drop-Off Point Box,Anne McLaren Building,2021-03-12 13:58:22.717000+00:00,Covid Testing,cu.covid.testing.1@gmail.com
...,...,...,...,...,...,...
90,JN-0-A,Delivery Box - Red,None,NaT,None,None
91,JN-0-B,Delivery Box - Red,None,NaT,None,None
112,LC-0-A,Delivery Box - Red,None,NaT,None,None
113,LC-0-B,Delivery Box - Red,None,NaT,None,None


In [19]:
df['box_pair'] = df['name'].str[:-2]
df['box_week'] = df['name'].str[-1]
df

,name,collection,location,last_seen_dt,last_seen_by,last_seen_by_email,box_pair,box_week
0,CAI-1-A,Delivery Box - Red,Gonville & Caius College,2021-03-10 13:07:38.250000+00:00,Covid Testing,cu.covid.testing.4@gmail.com,CAI-1,A
1,CAI-1-B,Delivery Box - Red,Box Storage Room,2021-03-11 09:41:30.471000+00:00,Covid Testing,cu.covid.testing.4@gmail.com,CAI-1,B
2,CAI-2-A,Delivery Box - Red,Gonville & Caius College,2021-03-10 13:07:40.699000+00:00,Covid Testing,cu.covid.testing.4@gmail.com,CAI-2,A
3,CAI-2-B,Delivery Box - Red,None,NaT,None,None,CAI-2,B
4,CAI-3-A,Delivery Box - Red,Gonville & Caius College,2021-03-10 13:07:42.651000+00:00,Covid Testing,cu.covid.testing.4@gmail.com,CAI-3,A
...,...,...,...,...,...,...,...,...
190,WY-P-1-C,Delivery Box - Red,Box Storage Room,2021-01-18 13:01:33.723000+00:00,becca Clarke,rjc204@cam.ac.uk,WY-P-1,C
191,WY-P-1-D,Delivery Box - Red,Box Storage Room,2021-01-18 13:01:13.839000+00:00,becca Clarke,rjc204@cam.ac.uk,WY-P-1,D
192,WY-P-1-E,Delivery Box - Red,Box Storage Room,2021-01-13 10:07:56.822000+00:00,Covid Testing,cu.covid.testing.4@gmail.com,WY-P-1,E
193,WY-P-2-A,Drop-Off Point Box,None,2020-11-12 10:48:46.590000+00:00,Rob Glew,rg522@cam.ac.uk,WY-P-2,A


In [24]:
df_lup = pd.read_excel("UMS_Courier_Schedules.xlsx", sheet_name='Box_Group_Lookup')
df_lup

,Collection,Box,Group
0,Delivery Box - Red,HO-1,R1
1,Delivery Box - Red,HO-2,R1
2,Delivery Box - Red,PET-1,R1
3,Delivery Box - Red,PEM-1,R1
4,Delivery Box - Red,PEM-2,R1
...,...,...,...
68,Drop-Off Point Box,N-P-1,D1
69,Drop-Off Point Box,ENG-P-1,D1
70,Drop-Off Point Box,ENG-P-2,D1
71,Drop-Off Point Box,H-P-1,D1


In [26]:
df_sch = pd.read_excel("UMS_Courier_Schedules.xlsx", sheet_name='Group_Schedule')
df_sch

,Group,Monday-A,Tuesday-A,Wednesday-A,Thursday-A,Friday-A,Saturday-A,Sunday-A,Monday-B,Tuesday-B,Wednesday-B,Thursday-B,Friday-B,Saturday-B,Sunday-B
0,R1-A,AMB,WFB,WFB,WFB,WFB,WFB,WFB,Colleges,Colleges,Colleges,Colleges,Colleges,Colleges,Colleges
1,R2-A,Colleges,AMB,WFB,WFB,WFB,WFB,WFB,WFB,Colleges,Colleges,Colleges,Colleges,Colleges,Colleges
2,R3-A,Colleges,Colleges,AMB,WFB,WFB,WFB,WFB,WFB,WFB,Colleges,Colleges,Colleges,Colleges,Colleges
3,R1-B,Colleges,Colleges,Colleges,Colleges,Colleges,Colleges,Colleges,AMB,WFB,WFB,WFB,WFB,WFB,WFB
4,R2-B,WFB,Colleges,Colleges,Colleges,Colleges,Colleges,Colleges,Colleges,AMB,WFB,WFB,WFB,WFB,WFB
5,R3-B,WFB,WFB,Colleges,Colleges,Colleges,Colleges,Colleges,Colleges,Colleges,AMB,WFB,WFB,WFB,WFB
6,D1-A,AMB,DO,AMB,DO,AMB,AMB,AMB,DO,AMB,DO,AMB,DO,DO,DO
7,D1-B,DO,AMB,DO,AMB,DO,DO,DO,AMB,DO,AMB,DO,AMB,AMB,AMB
8,D2-A,AMB,DO,AMB,DO,DO,DO,DO,AMB,DO,AMB,DO,DO,DO,DO
9,D2-B,DO,AMB,DO,AMB,AMB,AMB,AMB,DO,AMB,DO,AMB,AMB,AMB,AMB
